In [1]:
from openbb_terminal.sdk import openbb
from financialmodelingprep import fmp_stocks
import yaml
import pandas as pd
from datetime import datetime
from db.schema_models import LogAvailableTickers, LogIngestedTickers
from db.db_utils import insert_data, read_table_data
from db.db_engine import engine

with open('credentials.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [2]:
# Fetch the companies and tickers
openbb.keys.fmp(key=config['api']['fmp'], persist=True)
companies = fmp_stocks.available_companies(config['api']['fmp'])
companies_nasdaq = companies[(companies['exchangeShortName'] == "NASDAQ") & (companies['type'] == "stock")]
list_ticker = companies_nasdaq.symbol.to_list()

In [5]:
import qgrid

ModuleNotFoundError: No module named 'qgrid'

In [4]:
companies

,symbol,name,price,exchange,exchangeShortName,type
0,PMGOLD.AX,Perth Mint Gold,17.9400,Australian Securities Exchange,ASX,etf
1,ATEC,"Alphatec Holdings, Inc.",15.0150,NASDAQ Global Select,NASDAQ,stock
2,PEP,"PepsiCo, Inc.",184.6231,NASDAQ Global Select,NASDAQ,stock
3,TSLA,"Tesla, Inc.",244.3101,NASDAQ Global Select,NASDAQ,stock
4,KD,"Kyndryl Holdings, Inc.",14.9943,New York Stock Exchange,NYSE,stock
...,...,...,...,...,...,...
73041,HOFBF,Hofseth BioCare ASA,1.0000,Other OTC,PNK,stock
73042,LESG.L,Lyxor MSCI EM ESG Leaders Extra UCITS ETF,14.3020,London Stock Exchange,LSE,etf
73043,NCA.L,New Century AIM VCT Plc,25.5000,London Stock Exchange,LSE,stock
73044,MERCATOR.NS,Mercator Limited,0.9000,National Stock Exchange of India,NSE,stock


In [3]:
# Insert the data into the database
record = {
    'timestamp': datetime.now(),
    'num_companies': len(companies),
    'num_tickers': len(list_ticker)
}
insert_data(LogAvailableTickers, record)

In [4]:

# Partiamo da una lista di ticker ridotta e iteriamo su 100 trimestri. Sicuramente c'è un modo piu ottimale di farlo, ma al momento lo lasciamo cosi
list_ticker = ['AAPL', "PLTR","AMZN","META","NIO","RGTI","FFIEW","NVDA","NFLX"]

source="FinancialModelingPrep"
quarterly=True
limit=100

# Leggiamo i dati già storicizzati
df = read_table_data(engine, "fa_balance")   

In [5]:
for c,ticker in enumerate(list_ticker):
    
    # Per ogni ticker leggiamo i dati del bilancio trimestrale
    
    df_quarterly = openbb.stocks.fa.balance(symbol=ticker, source=source, quarterly=quarterly, limit=limit)
    df_quarterly = df_quarterly.T.reset_index()
    
    
    # Aggiungiamo la colonna della data corrente e quella del ticker in quanto non è presente
    df_quarterly['ticker'] = ticker
    df_quarterly['current_date'] = pd.Timestamp.now().floor('S')
    
    df_quarterly.columns = [col.lower().replace(" ","_") for col in df_quarterly.columns]
    
    
    df['calendar_year'] = pd.to_datetime(df['calendar_year'] )
    df_quarterly['calendar_year'] =pd.to_datetime(df_quarterly['calendar_year'] )

    # Prendiamo solo i record che non sono già presenti rispetto a Cik, Calendar year, Period
    
    if df.empty:
        df = df_quarterly.copy()
        
        record = {
            'ingestion_timestamp': datetime.now(),
            'ticker': ticker,
            'num_new_rows': len(df),
            'num_old_rows_diff_dates': 0,
            'num_columns': len(df.columns)
        }
        
    else:
        df_quarterly_merge = df_quarterly.merge(df, how='left', indicator=True, on=['cik', 'calendar_year', 'period'], suffixes=('', '_y'))
        df_new_rows = df_quarterly_merge[df_quarterly_merge['_merge'] == 'left_only']
        
        df = df.append(df_new_rows[df.columns])
            
        # Verifichiamo se ci sono record che sono già presenti ma con date diverse
        df_old_rows = df_quarterly_merge[df_quarterly_merge['_merge'] == 'both']
        
        df_old_rows_diff_dates = df_old_rows[(df_old_rows['filling_date'] != df_old_rows['filling_date_y']) | (df_old_rows['accepted_date'] != df_old_rows['accepted_date_y'])]
        df = df.append(df_old_rows_diff_dates[df.columns])


    # Creiamo e inseriamo il nuovo record in log_ingested_tickers
        record = {
            'ingestion_timestamp': datetime.now(),
            'ticker': ticker,
            'num_new_rows': len(df_new_rows),
            'num_old_rows_diff_dates': len(df_old_rows_diff_dates),
            'num_columns': len(df_quarterly.columns)
        }
        
    insert_data(LogIngestedTickers, record)



date_columns = ['filling_date', 'accepted_date', 'current_date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col])
    
df.to_sql('fa_balance', engine, if_exists='replace', index=False)


# Sicuramente c'è un modo più elegante per fare questo, ma per ora va bene così. L'obiettivo è mettere sul tavolo un primo modello il piu presto possibile e strutturare e consolidare l'indea, la pipeline e il codice man mano


0 AAPL
1 PLTR
2 AMZN
3 META
4 NIO
5 RGTI
6 FFIEW
7 NVDA
8 NFLX


481